In [ ]:

  import json
import csv
import glob

import collections



finalList = []
extension = 'json'
def writeTweetData():
    fileName='MedicalDyfunctions'
    with open('%s.csv' % (fileName), 'w') as file:
        w = csv.writer(file)
        w.writerow(['CUID', 'Concept', 'AuthorID', 'TweetText', 'CreatedDate', 'HashTags', 'Language'])
        all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
        for eachFile in all_filenames:
            dyfunctionfile = open(eachFile)
            jsonList = json.load(dyfunctionfile)
            global dataList
            for eachJson in jsonList:
                cui = eachJson['cui']
                concept = eachJson['concepts']
                tweet_data = eachJson['tweets']['data']
                for each_tweet_data in tweet_data:
                    author_id = each_tweet_data['author_id']
                    text = each_tweet_data['text']
                    lang = each_tweet_data['lang']
                    created_at = each_tweet_data['created_at']
                    tweet_data_dic = {
                    "CUID": cui,
                    "Concept": concept,
                    "AuthorID": author_id,
                    "TweetText": text,
                    "Language":lang,
                    "CreatedDate": created_at
                     }
                    if 'en' in lang:
                        if len(finalList) != 0:
                            exists = is_tweet_exists(finalList,tweet_data_dic)
                            if not exists:
                                hashTags = get_hashtags(text)
                                if hashTags is not None:
                                    tweet_data_dic["HashTags"]=hashTags
                                else:
                                    tweet_data_dic["HashTags"]=''

                                finalList.append(tweet_data_dic)
                                w.writerow([tweet_data_dic['CUID'], tweet_data_dic['Concept'],tweet_data_dic['AuthorID'],tweet_data_dic['TweetText'],tweet_data_dic['CreatedDate'],tweet_data_dic['HashTags'], tweet_data_dic['Language']])
                        else:
                            hashTags = get_hashtags(text)
                            if hashTags is not None:
                                tweet_data_dic["HashTags"]=hashTags
                            else:
                                tweet_data_dic["HashTags"]=''

                            finalList.append(tweet_data_dic)
                            w.writerow([tweet_data_dic['CUID'], tweet_data_dic['Concept'],tweet_data_dic['AuthorID'],tweet_data_dic['TweetText'],tweet_data_dic['CreatedDate'],tweet_data_dic['HashTags'], tweet_data_dic['Language']])


def is_tweet_exists(finalList, tweet_data_dic):
    for dict in finalList:
        if (tweet_data_dic['CUID'] == dict['CUID']) and (tweet_data_dic['AuthorID'] == dict['AuthorID']) and (tweet_data_dic['TweetText'] == dict['TweetText']) and (tweet_data_dic['CreatedDate'] == dict['CreatedDate']):
            return True
    return False

def get_hashtags(text):
    hashTagSet=set()
    hashtag_list = []
    hashtags = ''
    if '#' in text:
        for word in text.split():
            if word[0] == '#':
                hashtag_list.append(word[1:])
                hashTagSet= set(hashtag_list)
                hashtags = ', '.join(hashTagSet)
        return hashtags
    return None

writeTweetData()



In [6]:
import pandas as pd
df = pd.read_csv(r"MedicalDyfunctions.csv")

df



,CUID,Concept,AuthorID,TweetText,CreatedDate,HashTags,Language
0,C0018520,Halitosis,1356033810568110080,@MeidasTouch Pay-per-view?\nYou don't have eno...,2021-06-25T17:29:57.000Z,Halitosis,en
1,C0018520,Halitosis,1135548202411790337,A proven solution to naturally eliminate tonsi...,2021-06-24T15:16:21.000Z,"TonsilStones, Halitosis, oralcare",en
2,C0018520,Halitosis,1186392867209728000,The #GenuineHalitosis coming from the #mouth i...,2021-06-24T12:04:32.000Z,"mouth, halitosis,, GenuineHalitosis",en
3,C0018520,Halitosis,1931004288,#Dentist #Halitosis #Teeth\nHow To Get Rid Of ...,2021-06-24T10:51:01.000Z,"Dentist, Teeth, Halitosis",en
4,C0018520,Halitosis,1186392867209728000,How to Free Yourself From #Halitosis Forever! ...,2021-06-24T08:03:39.000Z,"badbreathcure, Halitosis",en
...,...,...,...,...,...,...,...
1240839,C0232841,Bladder dysfunction,216650914,RT @Cath_Browne: Detailed talk about #neurogen...,2014-02-09T16:15:10.000Z,"MS, neurogenicbladder, bladderdysfunction",en
1240840,C0232841,Bladder dysfunction,1382656436,Detailed talk about #neurogenicbladder to incr...,2014-02-09T13:27:11.000Z,"MS, neurogenicbladder, bladderdysfunction",en
1240841,C0232841,Bladder dysfunction,516480825,RT @MariaBGarrett: Blaithin speaks about our r...,2013-09-16T13:43:11.000Z,"Multiplesclerosis, bladderdysfunction",en
1240842,C0232841,Bladder dysfunction,824750786,Blaithin speaks about our research #bladderdys...,2013-09-16T12:39:57.000Z,"Multiplesclerosis, bladderdysfunction",en


In [7]:
import re

# Reading emoji patterns which contains emoticons , symbols & pictographs, transport & map symbols , flags
emojisCollection = re.compile("["
                       u"\U0001F600-\U0001F64F"  
                       u"\U0001F300-\U0001F5FF"  
                       u"\U0001F680-\U0001F6FF"  
                       u"\U0001F1E0-\U0001F1FF"  
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
df['CleanedTweet'] = (df['TweetText'].str.lower().str.replace(r'[^\w\s]+', '').str.replace(emojisCollection, '').str.strip())
df['CleanHashTags'] = (df['HashTags'].str.lower().str.replace(r'[^\w\s]+', ',').str.replace(emojisCollection, '').str.strip())

In [9]:
dataFrame=df[['CUID','Concept','AuthorID','TweetText','CleanedTweet','CreatedDate','HashTags','CleanHashTags','Language']]
dataFrame.to_csv('CleanedMedicalConcepts.csv')


In [10]:
dataFrame

,CUID,Concept,AuthorID,TweetText,CleanedTweet,CreatedDate,HashTags,CleanHashTags,Language
0,C0018520,Halitosis,1356033810568110080,@MeidasTouch Pay-per-view?\nYou don't have eno...,meidastouch payperview\nyou dont have enough t...,2021-06-25T17:29:57.000Z,Halitosis,halitosis,en
1,C0018520,Halitosis,1135548202411790337,A proven solution to naturally eliminate tonsi...,a proven solution to naturally eliminate tonsi...,2021-06-24T15:16:21.000Z,"TonsilStones, Halitosis, oralcare","tonsilstones, halitosis, oralcare",en
2,C0018520,Halitosis,1186392867209728000,The #GenuineHalitosis coming from the #mouth i...,the genuinehalitosis coming from the mouth is ...,2021-06-24T12:04:32.000Z,"mouth, halitosis,, GenuineHalitosis","mouth, halitosis, genuinehalitosis",en
3,C0018520,Halitosis,1931004288,#Dentist #Halitosis #Teeth\nHow To Get Rid Of ...,dentist halitosis teeth\nhow to get rid of bad...,2021-06-24T10:51:01.000Z,"Dentist, Teeth, Halitosis","dentist, teeth, halitosis",en
4,C0018520,Halitosis,1186392867209728000,How to Free Yourself From #Halitosis Forever! ...,how to free yourself from halitosis forever ht...,2021-06-24T08:03:39.000Z,"badbreathcure, Halitosis","badbreathcure, halitosis",en
...,...,...,...,...,...,...,...,...,...
1240839,C0232841,Bladder dysfunction,216650914,RT @Cath_Browne: Detailed talk about #neurogen...,rt cath_browne detailed talk about neurogenicb...,2014-02-09T16:15:10.000Z,"MS, neurogenicbladder, bladderdysfunction","ms, neurogenicbladder, bladderdysfunction",en
1240840,C0232841,Bladder dysfunction,1382656436,Detailed talk about #neurogenicbladder to incr...,detailed talk about neurogenicbladder to incre...,2014-02-09T13:27:11.000Z,"MS, neurogenicbladder, bladderdysfunction","ms, neurogenicbladder, bladderdysfunction",en
1240841,C0232841,Bladder dysfunction,516480825,RT @MariaBGarrett: Blaithin speaks about our r...,rt mariabgarrett blaithin speaks about our res...,2013-09-16T13:43:11.000Z,"Multiplesclerosis, bladderdysfunction","multiplesclerosis, bladderdysfunction",en
1240842,C0232841,Bladder dysfunction,824750786,Blaithin speaks about our research #bladderdys...,blaithin speaks about our research bladderdysf...,2013-09-16T12:39:57.000Z,"Multiplesclerosis, bladderdysfunction","multiplesclerosis, bladderdysfunction",en


In [11]:
df_new = df[df['CleanHashTags'].notnull()]
conceptFrequency = {}
def freqCount():
  for hashtag in df_new["CleanHashTags"]:
    strToken = hashtag.split(', ')
    for token in strToken:
      if token not in conceptFrequency.keys():
        conceptFrequency[token] = 1
      else:
        conceptFrequency[token] = conceptFrequency[token]+1
  return conceptFrequency

freqCount()

df = pd.DataFrame(list(conceptFrequency.items()),columns = ['Concept','Frequency']) 
df= df.sort_values(by='Frequency', ascending=False, ignore_index=True)
df

,Concept,Frequency
0,health,19828
1,covid19,16126
2,,12324
3,diabetes,12041
4,cancer,10478
...,...,...
315489,hypogastricplexus,1
315490,antidepresssants,1
315491,eddrugs,1
315492,edmedications,1
